# Week4 - 銀行顧客の定期預金申し込みを予測してみよう!


## 前回のおさらい
機械学習のステップは以下でした。
![おさらい](./images/おさらい.png)



ちょっと待った！　訓練データ　と　検証データは分けるんでしたよね！！
![traintest](./images/traintest.png)

## 学習の目標

銀行顧客の定期預金申し込みを予測、活用するCRISP-DMのサイクルを回してみる。
決定木モデルを作成し、予測結果を評価する
ハイパーパラメータのチューニングをする（グリッドサーチの技法）方法を学ぶ
予測結果を可視化し活用するダッシュボードを作ってみる


## 1.データの理解
bank-additional-full.csv に接続します。Tableau Desktopから接続し、どのような分類で申し込み＝Yが多いかあたりをつけましょう。


![deta_list](./images/data_list.png)

In [18]:
%%HTML
<div class='tableauPlaceholder' id='viz1580558060146' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Z6&#47;Z6ZG6CZBX&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;Z6ZG6CZBX' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Z6&#47;Z6ZG6CZBX&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1580558060146');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## 2.予測モデルの作成とチューニング

それでは、早速予測モデルを作成していきしょう。ここからはJupyter Notebookを利用します。
今回は決定木をPythonのscikit-learnに含まれるDecisionTree Classifier（決定木モデル）を利用します。必要なライブラリを読み込みます。

## 本日の流れを確認します。
![week4overview](./images/week4-overview.jpg)

In [1]:
##import を実施します。

import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss , accuracy_score

In [20]:
##初めの一回はpipでインストールが必要です。　決定木グラフの構造を書き出す際に利用します。

##!pip install pydotplus

先ほどの訓練データを学習して、モデルを作成します。まずは訓練データを読み込みます。

In [21]:
##データを読み込みます。
#df = pd.read_csv("bank-additional-full.csv", delimiter=";")

読み込んだデータを確認します。

In [2]:
##データを確認します。
#df.head()


訓練データの行数と列数を確認します。

In [3]:
#df.shape


入力変数と出力変数をそれぞれ抜き出します。

In [24]:
##目的変数を抜き出します。　やってみましょう！
#t = 

##説明変数を抜き出します。　やってみましょう！
#x = 


### ダミー変数化
カテゴリカルな（連続した数値ではなく、文字列を含む）データはそのままではモデル作成に利用できません！

ですのでダミー変数化(カテゴリごとに1または0の値を持つフィールドに分割）します。
![dummy](./images/dummy.png)

In [4]:
##ダミー変数化をおこないます 
#xd = pd.get_dummies(x)


内容を確認します。

In [5]:
##内容を確認します。
#xd.head()


### 訓練データと検証データの分割
訓練データと検証データに分割します。ここでは、test_size=0.1 として全体の1割を検証データに利用します。

In [27]:
## 訓練データと検証データに分割します。   
from sklearn.model_selection import train_test_split
x_train, x_val, t_train, t_val = train_test_split(xd, t, test_size=0.1, random_state=1)

In [6]:
##訓練データの行数を確認します。
#len(x_train)

In [7]:
##検証データの行数を確認します。
#len(x_val)

### モデルの作成と学習 max_depth=3

決定木モデルの作成時に、ハイパーパラメータを設定することができます、ここではmax_depth=3、min_samples_leaf=500 を設定します。続けて、訓練データをモデルに代入し学習（fit）させます。

In [31]:
## max_depth =3 でモデルの作成
# clf3 = DecisionTreeClassifier(max_depth=3,min_samples_leaf=500)

clf3 = DecisionTreeClassifier(max_depth=3, min_samples_leaf=500)


In [8]:
## 訓練データをモデルに代入し学習（fit）させます。
#clf3.fit(x_train,t_train)

学習させたモデルを 「Web Graphviz 」ツールにて可視化します。
※　Jupyter Notebook内に決定木モデルの可視化を表示する場合は、「Tableauで始めるデータサイエンス巻末、補足資料「graphvizツール導入手順」を参照ください。


In [33]:
## max_depth =3 で決定木モデルのグラフ化
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

export_graphviz(clf3, out_file="tree_clf3.dot", feature_names=xd.columns, class_names=["0","1"], filled=True, rounded=True)
g = pydotplus.graph_from_dot_file(path="tree_clf3.dot")

## GraphvizをインストールしてJupyterNotebook内に決定木グラフを表示する場合以下を実行します。
## Image(g.create_png())

### 決定木グラフの可視化手順
① Jupyter notebook を起動したディレクトリに "tree_clf3.dot" ファイルがあることを確認します。

② notepad等のエディタから①を開きます。

③ Ctrl+A ですべて選択しコピーします。

④ http://www.webgraphviz.com/　を開きます。

⑤コードを④にコピーし、Generate Graph!　をクリックします。

![clf3](./images/webgraphviz.png)

グラフの読み方としては、青色の濃い部分は目的変数である、「定額預金の申し込みが有（yes) 」が多い分類、逆にオレンジ色は少ない分類になっているということになります。青色が濃いところに注目すると、duration <=172.5 ではない（つまり duration > 172.5) かつ pdays <= 513.0 の区分の青色が濃いです。つまり、duration > 172.5　かつ　pdpoutcome_success>0.5 である場合に、申し込みが有る可能性が高いということになります。前回キャンペーンに成功していて、コンタクト（会話）時間が172.5秒以上の場合申し込みをする可能性が高いという事ですね。

MEMO: 「偏り」や「不均等さ」を数値で表したものが「ジニ係数」です。ジニ係数は0から1までの値をとり、1に近いほど偏りが大きく、0に近いほど偏りが小さいことを表します。

訓練データと検証データで精度を確認します。

In [9]:
## 訓練データでの精度 = 何個中何個当たっていたか？(Accuracy）を確認します。
#clf3.score(x_train, t_train) 

In [10]:
## 検証データでの精度 = 何個中何個当たっていたか？(Accuracy）を確認します。
#clf3.score(x_val, t_val)

訓練データと検証データそれぞれでモデルの精度（正解のレコード数 / 全体のレコード数）のスコアを確認します。訓練データ＝xx.xx%、検証データ＝xx.xx%となりました。

## 予測モデルのチューニングを実施

### モデルの作成と学習 max_depth=6
次にパラメータをチューニングして、モデルの精度が変化するかどうかを確認します。max_depth=6 に増やしてモデルを作成してみましょう。
ちなみに他にどのようなパラメータがあるのかについては、Notebookで「DecisionTreeClassifier」にカーソルを置いて「Shift+Tab」、＋ボタンを押して表示を拡張します。

In [36]:
## max_depth =6 でモデルの作成
# clf6 = DecisionTreeClassifier(max_depth=6 , min_samples_leaf=500)
clf6 = DecisionTreeClassifier(max_depth=6 , min_samples_leaf=500)


In [11]:
## 訓練データをモデルに代入し学習（fit）させます。
# clf6.fit(x_train,t_train)

In [38]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
export_graphviz(clf6, out_file="tree_clf6.dot", feature_names=xd.columns, class_names=["0","1"], filled=True, rounded=True)
g = pydotplus.graph_from_dot_file(path="tree_clf6.dot")

##Image(g.create_png())

maxdepth=3 の時と同様にWebGraphvizを使って maxdepth=6 での決定木グラフを書いてみましょう。
![clf6](./images/clf6.png)


In [12]:
## 訓練データでの精度 = 何個中何個当たっていたか？(Accuracy）を確認します。
#clf6.score(x_train, t_train)  

In [13]:
## 検証データでの精度 = 何個中何個当たっていたか？(Accuracy）を確認します。
#clf6.score(x_val, t_val)  # Accuracy: 精度 = 何個中何個当たっていたか？

### モデルの作成と学習 max_depth=12

In [41]:
##モデルの作成
clf12 = DecisionTreeClassifier(max_depth=12 , min_samples_leaf=500)
clf12.fit(x_train,t_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=500, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [42]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
export_graphviz(clf12, out_file="tree_clf12.dot", feature_names=xd.columns, class_names=["0","1"], filled=True, rounded=True)
g = pydotplus.graph_from_dot_file(path="tree_clf12.dot")
##Image(g.create_png())

maxdepth=3 の時と同様にWebGraphvizを使って maxdepth=12 での決定木グラフを書いてみましょう。
![clf6](./images/clf12.png)

In [14]:
## 訓練データでの精度 = 何個中何個当たっていたか？(Accuracy）を確認します。
#clf12.score(x_train, t_train)  # Accuracy: 精度 = 何個中何個当たっていたか？

In [15]:
## 検証データでの精度 = 何個中何個当たっていたか？(Accuracy）を確認します。
#clf12.score(x_val, t_val)  # Accuracy: 精度 = 何個中何個当たっていたか？

### クロスバリデーションとは？

今までの学習法では、検証データは学習に利用していませんでした、これってもったいなくないですか？
更に、たまたま、学習に使ったデータが偏っていたらどうでしょうか？少し不安ですね。

ここで、クロスバリデーションの登場です。K-分割交差検証（K-fold cross-validation）とも言います。

　K- 分割交差検証とは、今までのようにある1つの訓練データ、検証データに対し
学習から評価までを行うのではなく、 K個のデータに分割後、 1個の検証データとK-1個の訓練データに対し、K 回検証を行う方法になります。

　 K=5の場合を考えてみましょう。まず5 個のデータに分割し、そのうちの1つのデータを検証データ、残り4つのデータを訓練データにして学習から評価までを行います。
これが図①にあたります。次に、①で検証データに使用しなかったデータ
の内の1つを検証データ、残りを訓練データに使用し、学習から評価までを行います
（②）。この流れを、全ての分割データを検証データに使用するまで繰り返します。つまり、今回の場合は5 回繰り返すことになります。最終的に全ての検証スコアを合わせて評価します。

[参考](https://qiita.com/LicaOka/items/c6725aa8961df9332cc7 "比較的少なめのデータで機械学習する時は交差検証 (Cross Validation) をするのです")
![交差検証](./images/交差検証.png)

### 最適なハイパーパラメータの調整（Grid Searchの利用）

ここまでmax_depthの大きさを3、6、12と変更して確認を行いましたが、他の値ではどうだろう？その他のパラメータを変更した場合はどうなるだろう・・・一体どのパラメータの組み合わせが最適なのだろうと疑問を持ちますね。

しかし、パラメータの組み合わせは膨大にあり一つ一つ確認するのは気の遠くなる作業となります。

そこで、**「グリッドサーチ」** という方法が役立ちます。

「グリッドサーチ」では検証したいパラメータのリストを指定し、総当たりの組み合わせでモデルの精度を検証し、最も当てはまりの良い組み合わせを「best parameter」として提示してくれます。

更に内部で前述の**クロスバリデーション**もやってくれます。
![GridSearch](./images/gridsearch.png)

早速やってみましょう。

ここでは検証の対象となるパラメータを
max_depth を [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]と
min_sample_leaf を[100,200,500,1000]と指定します。

sklearnの中のmodel_selectionからGridSearchCVをimportしましょう。

In [45]:
## モジュールのインポート
from sklearn.model_selection import GridSearchCV

Grid SearchCVを用いる際には、下記3つを用意する必要があります。

 - estimator：使用する手法
 - param_grid：ハイパーパラメータの探索範囲
 - cv：データの分割数

In [46]:
## DecisionTreeClasifirerを使用
#estimator =  DecisionTreeClassifier()


#### 決定木のハイパーパラーメータ

決定木の主要なハイパーパラーメータとしては以下があります。

- max_depth	決定木の深さの最大、過学習を抑止する効果がある
- min_samples_leaf　1つのノードに必要な最小のサンプル数

※　詳細についてはsklearnの説明ページを参照ください。
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

上記2つのハイパーパラーメータの探索範囲として以下のよう値を選択します。
各ハイパーパラーメータを辞書の**キー**として扱い、辞書の**値**に探索範囲をリスト形式で入れます。

In [47]:
## チューニングするパラメータ
param_grid = {'max_depth':  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'min_samples_leaf': [100,200,500,1000]
                   }

クロスバリデーションを5分割しましょう。

In [48]:
#cv = 5


`GridSearchCV`を用いると、 `train` と `validation` は自動的に分割されます。 

準備した各値を`GridSearchCV`に入れます。

In [49]:
## モデルの定義
#tuned_model = GridSearchCV(estimator = estimator , param_grid=param_grid, cv=cv, return_train_score=False)


通常通り、`fit()`で学習させましょう。 **少し時間がかかります。少々お待ちください。**

In [16]:
## モデルの学習 ※少し時間がかかります。少々お待ちください。
# tuned_model.fit(x_train, t_train)
#tuned_model.fit(x_train,t_train)

学習後、tuned_modelの中のcv_results_にクロスバリデーションを行った結果が入っていますので、pandasを用いて表形式で表示させてみましょう。

In [56]:
## モデルの検証結果をデータフレームにします。
df_cv = pd.DataFrame(tuned_model.cv_results_)
## 見にくいので行列転置します。
df_cv.T

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
mean_fit_time,0.0536486,0.053662,0.0522643,0.0540466,0.0674244,0.0682091,0.0686257,0.0676203,0.0859686,0.0831556,...,0.132466,0.122463,0.15877,0.148822,0.13783,0.131654,0.185108,0.161781,0.14422,0.132652
std_fit_time,0.00316379,0.00144791,0.000795888,0.00171098,0.00136853,0.00173782,0.00158627,0.00147548,0.00405995,0.00202438,...,0.00182668,0.00202808,0.00442885,0.00146682,0.00212412,0.00260579,0.0140813,0.00846145,0.00896462,0.0103316
mean_score_time,0.013971,0.0125625,0.0125673,0.0121716,0.0127698,0.0133773,0.0129554,0.0127699,0.0129713,0.0129704,...,0.0123556,0.012975,0.013168,0.0127697,0.0129764,0.0143456,0.0155639,0.013356,0.013369,0.0133659
std_score_time,0.00302293,0.0004927,0.000487996,0.000396116,0.000396756,0.0013506,0.000616432,0.000399845,0.00156049,0.00106746,...,0.00049485,0.000639762,0.000393695,0.000394402,1.68165e-05,0.00172875,0.00324794,0.00101393,0.00148782,0.00183429
param_max_depth,1,1,1,1,2,2,2,2,3,3,...,8,8,9,9,9,9,10,10,10,10
param_min_samples_leaf,100,200,500,1000,100,200,500,1000,100,200,...,500,1000,100,200,500,1000,100,200,500,1000
params,"{'max_depth': 1, 'min_samples_leaf': 100}","{'max_depth': 1, 'min_samples_leaf': 200}","{'max_depth': 1, 'min_samples_leaf': 500}","{'max_depth': 1, 'min_samples_leaf': 1000}","{'max_depth': 2, 'min_samples_leaf': 100}","{'max_depth': 2, 'min_samples_leaf': 200}","{'max_depth': 2, 'min_samples_leaf': 500}","{'max_depth': 2, 'min_samples_leaf': 1000}","{'max_depth': 3, 'min_samples_leaf': 100}","{'max_depth': 3, 'min_samples_leaf': 200}",...,"{'max_depth': 8, 'min_samples_leaf': 500}","{'max_depth': 8, 'min_samples_leaf': 1000}","{'max_depth': 9, 'min_samples_leaf': 100}","{'max_depth': 9, 'min_samples_leaf': 200}","{'max_depth': 9, 'min_samples_leaf': 500}","{'max_depth': 9, 'min_samples_leaf': 1000}","{'max_depth': 10, 'min_samples_leaf': 100}","{'max_depth': 10, 'min_samples_leaf': 200}","{'max_depth': 10, 'min_samples_leaf': 500}","{'max_depth': 10, 'min_samples_leaf': 1000}"
split0_test_score,0.888454,0.888454,0.888454,0.888454,0.905584,0.905584,0.905584,0.905584,0.911789,0.911789,...,0.915295,0.911789,0.915295,0.913272,0.915295,0.911789,0.915295,0.913272,0.915295,0.911789
split1_test_score,0.888454,0.888454,0.888454,0.888454,0.905719,0.905719,0.905719,0.905719,0.907877,0.907877,...,0.913947,0.907877,0.912598,0.912193,0.913947,0.907877,0.912058,0.912193,0.913947,0.907877
split2_test_score,0.888454,0.888454,0.888454,0.888454,0.902752,0.902752,0.902752,0.902752,0.908282,0.908282,...,0.912868,0.908282,0.911789,0.914351,0.912868,0.908282,0.911789,0.914351,0.912868,0.908282


In [57]:
##データフレームにはquery()メソッドが用意されていているので、スコアランクが高いものにフィルタします。
df_cv.query('rank_test_score <= 10').T

,12,16,17,18,20,22,26,30,34,38
mean_fit_time,0.0987337,0.130656,0.121077,0.109508,0.127693,0.117803,0.127674,0.132466,0.13783,0.14422
std_fit_time,0.00258831,0.0132774,0.00949507,0.00525503,0.00122496,0.00229325,0.00340255,0.00182668,0.00212412,0.00896462
mean_score_time,0.0121858,0.0149538,0.0129653,0.0129645,0.0130084,0.0125727,0.0127667,0.0123556,0.0129764,0.013369
std_score_time,0.000407097,0.00141055,0.000892924,0.0006306,0.000654083,0.000491509,0.000390721,0.00049485,1.68165e-05,0.00148782
param_max_depth,4,5,5,5,6,6,7,8,9,10
param_min_samples_leaf,100,100,200,500,100,500,500,500,500,500
params,"{'max_depth': 4, 'min_samples_leaf': 100}","{'max_depth': 5, 'min_samples_leaf': 100}","{'max_depth': 5, 'min_samples_leaf': 200}","{'max_depth': 5, 'min_samples_leaf': 500}","{'max_depth': 6, 'min_samples_leaf': 100}","{'max_depth': 6, 'min_samples_leaf': 500}","{'max_depth': 7, 'min_samples_leaf': 500}","{'max_depth': 8, 'min_samples_leaf': 500}","{'max_depth': 9, 'min_samples_leaf': 500}","{'max_depth': 10, 'min_samples_leaf': 500}"
split0_test_score,0.915161,0.916374,0.915161,0.915295,0.917723,0.915295,0.915295,0.915295,0.915295,0.915295
split1_test_score,0.910709,0.912328,0.913272,0.913947,0.911519,0.913947,0.913947,0.913947,0.913947,0.913947
split2_test_score,0.912733,0.914891,0.912733,0.912868,0.913137,0.912868,0.912868,0.912868,0.912868,0.912868


実際に表を見なくても`best_params_`という値に最適なハイパーパラーメータが入っているので確認しましょう。

In [58]:
## 最適なハイパーパラメータの確認
tuned_model.best_params_

{'max_depth': 5, 'min_samples_leaf': 100}

`best_estimator_`には最適なハイパーパラーメータを使用した最適なモデルが入っています。最後に、こちらを用いてテストデータの結果も確認しましょう。

In [17]:
## 最適なモデルの引き継ぎ
#clf_best = tuned_model.best_estimator_

In [18]:
## モデルの内容を確認します。
#clf_best

モデルのテストを行います。まずはモデル作成に利用したtain_val(訓練＋検証）について

In [19]:
## モデルのテスト 
#print(clf_best.score(x_train, t_train))


モデルのテストを行います。次にモデル作成に利用しなかったtest(テスト）について

In [20]:
#print(clf_best.score(x_val, t_val))

訓練＋検証、テスト共に良い結果が得られているようです。チューニングされた決定木も可視化してみましょう。

In [63]:
## clf_bestを利用した決定木グラフの作成

from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
export_graphviz(clf_best, out_file="tree_clf_best.dot", feature_names=xd.columns, class_names=["0","1"], filled=True, rounded=True)
g = pydotplus.graph_from_dot_file(path="tree_clf_best.dot")
## Image(g.create_png())

maxdepth=3 の時と同様にWebGraphvizを使って clf_best での決定木グラフを書いてみましょう。
![clf_best](./images/clf_best.png)

それぞれのモデルについてtrainとvalのスコアを比較します。


## 4.未知のデータに対する予測を実施する


作成されたモデルを使って、結果の分からない全く新しいデータに対して予測を実施してみましょう。

ここではあらかじめ用意された、630件の定期預金を申し込んだかどうかの分からない顧客のデータ"sample.csv”を読み込みます。右端に”ｙ”の列がないことがポイントです。sampleという名前のデータフレームに投入します。


In [64]:
#　やってみましょう！
sample  =  pd.

In [22]:
#sample.head()

idは予測モデルを作成するための入力変数として必要がないため削除して、その他の入力変数について、カテゴリカルな変数をダミー変数化します。

In [23]:
## id を削除する やってみましょう！
sample1 = 

新しいデータに対してダミー変数化を行います。

In [67]:
## 新しいデータに対してダミー変数化を行います。
#x = pd.get_dummies(sample1)

In [24]:
#x.head()

作成したモデルを使って予測値を取得します。グリッドサーチの結果、最適なモデルは clf_best というモデルで保存しています。prodect_proba で予測値を算出します。列＝１の方が「定期預金を契約する＝ｙ」の確率です。

In [25]:
## 列＝１の方が「定期預金を契約する＝ｙ」の確率です。
#clf_best.predict_proba(x)

「定期預金を契約する＝ｙ」の確率を y_new に代入します。

In [70]:
## 作成したモデルを利用してsample に対して予測値を取得します。
#y_new = clf_best.predict_proba(x)[:,1]


「定期預金を契約する＝ｙ」の確率でをデータフレームにし、列名として"predict"を付けます。

In [71]:
## 予測値をデータフレーム化します。
df_y_new = pd.DataFrame(y_new,  columns=['predict'])


一番初めに読み込んだsampleの右端に予測値を追加します。

In [72]:
## オリジナルのデータの右端に予測値を追加します。
sample_y_new = pd.concat([sample,df_y_new],axis=1)

中身を確認します。

In [26]:
#sample_y_new.head()

Tableau で予測値を利用した分析とダッシュボード作成を行うために、CSVファイルとして吐き出します。

In [74]:
## CSVファイルとして書き出します。
sample_y_new.to_csv("sample_y_new.csv")

### ここから先はTableau Desktopで確認します。

In [27]:
#%%HTML
#<div class='tableauPlaceholder' id='viz1580557814844' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;W3&#47;W359FJNH6&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;W359FJNH6' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;W3&#47;W359FJNH6&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1580557814844');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.85)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.95)+'px';} else { vizElement.style.width='100%';vizElement.style.minHeight='1150px';vizElement.style.maxHeight=(divElement.offsetWidth*1.77)+'px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### 参考資料

### 混合行列とは
予測値が閾値以上のものをY（正例）、以下のものを（負例）と判定したときに、実際にそれがあっている場合とあっていない場合があります。

![混合行列](./images/混合行列.png)

何を重要視するかで評価指標が異なります。
![混合行列の考え方](./images/accuracy.png)

![recall](./images/recall.png)

![Precision](./images/Precision.png)

※　F1値　も評価として利用されます
![F1](./images/f1.png)

**ROC曲線とは**
推定確立がρ以上のものを陽性とみなすとき、縦軸：真陽性率（TP/(TP+FN))、横軸：偽陽性率　(FP/(FP+TN)に対応する点を。推定確立ρに対してプロットしてつなげた折れ線グラフです・・・・・　（うーん、なんのこっちゃ）


![ROC](./images/ROC曲線.png)

**つまりこういうこと**

①　予測値が高い順からとりあえず並べる　
    →　本当にあってるものと、間違っちゃったFP（なんちゃってポジティブ）がある。
    
②　本当にあってれば、縦方向に上に進む（真陽性率上がる）間違えれば横に進む（偽陽性率上がる）

③　次に確率の低い点について確認、②を繰り返す

④　間違いなく、早く言い当てることができれば、上方向に進む、横方向には進まない。（→　これがいいモデル）

![ROC曲線の解釈](./images/ROC曲線の解釈.png)

※　正解から予測値がどれだけ離れいてるかの差を総和して評価する指標　logloss も指標として利用される場合があります。

機械学習でLog Lossとは何か
https://qiita.com/exp/items/1c6c9a3fae2d97bfa0c7
一言でいうと、クロスエントロピー。0~1の予測値を入力してモデルの性能を測る指標を出力する。

Accuracyとの違い
→　Accuracyは予測した値と正解が一致していた数のカウント。正解/不正解しかないのでいつも良い指標とは限らない（惜しかった、などが測れない）
→　Log Lossは実際のラベルからどのくらい違っていたのかを考慮できる


Kaggleで勝つデータ分析の技術より
![loggloss](./images/logloss.png)

### おまけ　Logistic回帰でやってみましょう。

In [88]:
#import
from sklearn.linear_model import LogisticRegression


In [89]:
#モデルの定義

model = LogisticRegression()
model.fit(x_train , t_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
model.score(x_train, t_train)  # Accuracy: 精度 = 何個中何個当たっていたか？

0.9101945021446491

In [91]:
model.score(x_val, t_val)  # Accuracy: 精度 = 何個中何個当たっていたか？

0.9050740470988103

## お疲れさまでした！